In [ ]:
from datetime import timedelta, datetime
import glob
import json
import os
import re

from collections import Counter

import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
val = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/val.json', typ = 'frame',encoding='UTF-8')
song = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/song_meta.json', typ = 'frame',encoding='UTF-8')
train = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/train.json', typ = 'frame', encoding='utf-8')
test = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/test.json', typ = 'frame', encoding='utf-8')
genre_gn_all = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/genre_gn_all.json', typ = 'series',encoding='utf-8')

## song 필터링

In [ ]:
train['num_songs'] = train['songs'].apply(len)
train.head(2)

,tags,id,plylst_title,songs,like_cnt,updt_date,num_songs
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,19
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,42


In [ ]:
%%time
# song_count 행렬 만들기
song_count = []
for i in train.songs.to_list():
  for j in i:
    song_count.append(j)
song_exist = pd.DataFrame(song_count)[0].value_counts().reset_index().rename(columns={'index':'song', 0:'count'})

CPU times: user 2.18 s, sys: 11.9 ms, total: 2.19 s
Wall time: 2.19 s


In [ ]:
song_exist.head()

,song,count
0,144663,2175
1,116573,2121
2,357367,1981
3,366786,1919
4,654757,1647


In [ ]:
song_exist['count'].describe() # 분포 쉐엣

count    615142.000000
mean          8.592928
std          39.481912
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max        2175.000000
Name: count, dtype: float64

노래 제외 X

In [ ]:
# # 애초에 3번 이하 등장하는 노래들은 제외하고 시작
# less_song = song_exist.query('count<2').song.values # 버릴애들
# only_song = song_exist.query('count>1').song.values # 얘네만 씀
# only_song_id = only_song.tolist()

# print('버릴 애들 : {}'.format(len(less_song)), '사용할 애들 : {}'.format(len(only_song))) 

In [ ]:
# new_song = song.query('id == @only_song_id') # 22만개만 씀
# print('새로운 song_meta 길이 : {}'.format(len(new_song))) # 새로운 song_meta

In [ ]:
# new_song.head(2) # 새로운 song meta

In [ ]:
# # 3번 이상 등장한 노래만 남기고 train 노래 삭제
# %%time
# # song_value = song_exist.query('count>2').song.values
# def song_del(x):
#   x_iter = []
#   for i in x:
#     if i in only_song:
#       x_iter.append(i)
#   return x_iter

# train['songs'] = train.songs.map(song_del)

In [ ]:
# # 이상한 날짜 제거
# song_meta = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/song_meta.json', typ = 'frame',encoding='UTF-8')
# plylst_date = train[['updt_date', 'songs']]
# plylst_date_unnest = np.dstack(
#     (
#         np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
#         np.concatenate(plylst_date.songs.values)
#     )
# )
# plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

# del plylst_date_unnest

# plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
# song_date = song_meta[["id","issue_date"]]
# plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
# plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

# plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
# strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)
# strange_songs = strange_songs.drop_duplicates(['id'])

# date_id = strange_songs.id.values

# def date_del(x):
#   x_iter = []
#   for i in x:
#     if i not in date_id:
#       x_iter.append(i)
#   return x_iter

# train['songs'] = train.songs.map(date_del)

In [ ]:
# train.loc[:, 'num_songs'] = train['songs'].map(len)
# print('train에 사용된 누적 song 개수 : {}'.format(train['num_songs'].sum())) # 오십만개 줄여서 사용

In [ ]:
# tag 인덱스 다시 부여
plylst_tag = train['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])
tag_dict = {x: tag_counter[x] for x in tag_counter}

tag_id_tid = dict()
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
    tag_id_tid[t] = i
    tag_tid_id[i] = t
n_tags = len(tag_dict)

# 새로운 tag id 매핑
train['tags_id'] = train['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

In [ ]:
# 사용 plylst
plylst_train = train[['plylst_title','updt_date','tags_id','songs']]
plylst_train.loc[:,'num_songs'] = plylst_train['songs'].map(len)
plylst_train.loc[:,'num_tags'] = plylst_train['tags_id'].map(len)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
# 사용된 song dict 구하기
plylst_song = train['songs']
song_dict = {x : x for x in song['id']}
print(len(song_dict))

707989


In [ ]:
n_train = len(plylst_train)

row = np.repeat(range(n_train), plylst_train['num_songs']) # User Index 별 노래 개수만큼 만듦 
col = [song for songs in plylst_train['songs'] for song in songs] # Song dic number 추출 
n_songs = max(col)+1
dat = np.repeat(1, plylst_train['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦

train_user_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs)) # csr_matrix 제작 
train_user_songs_A

<115071x707989 sparse matrix of type '<class 'numpy.longlong'>'
	with 5285871 stored elements in Compressed Sparse Row format>

In [ ]:
row = np.repeat(range(n_train), plylst_train['num_tags'])
col = [tag for tags in plylst_train['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_train['num_tags'].sum())
train_user_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_tags))

In [ ]:
train_user_songs_A_T = train_user_songs_A.T.tocsr()  ## csr matrix로 변환됨
train_user_tags_A_T = train_user_tags_A.T.tocsr() 
train_user_songs_A_T, train_user_tags_A_T

(<707989x115071 sparse matrix of type '<class 'numpy.longlong'>'
 	with 5285871 stored elements in Compressed Sparse Row format>,
 <29160x115071 sparse matrix of type '<class 'numpy.longlong'>'
 	with 476331 stored elements in Compressed Sparse Row format>)

## 장르, 제목 array

In [ ]:
# song 장르 가져오기. 
# song = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/song_meta.json', typ = 'frame',encoding='UTF-8')
song_cate = []

for i in range(len(train)):
    gnr = [] 
    songs = train.iloc[i,3]
    
    for j in songs:
        for k in song.iloc[j,0]:
            gnr.append(k)
    song_cate.append(gnr)
        
train['plylst_genre'] = song_cate

plylst_genre = train['plylst_genre']
genre_counter = Counter([gen for genre in plylst_genre for gen in genre])
genre_dict = {x: genre_counter[x] for x in genre_counter}

genre_id_tid = dict()
genre_tid_id = dict()
for i, t in enumerate(genre_dict):
    genre_id_tid[t] = i
    genre_tid_id[i] = t
n_genre = len(genre_dict)
train['plylst_genre_id'] = train['plylst_genre'].map(lambda x: [genre_id_tid.get(s) for s in x if genre_id_tid.get(s) != None])

In [ ]:
train.head(2)

,tags,id,plylst_title,songs,like_cnt,updt_date,num_songs,tags_id,plylst_genre,plylst_genre_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,19,[0],"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...","[0, 1, 2, 3, 4, 5, 6, 4, 7, 2, 3, 4, 7, 2, 3, ..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,42,"[1, 2]","[GN0101, GN0103, GN0601, GN0605, GN0104, GN010...","[18, 19, 20, 21, 22, 18, 20, 21, 22, 18, 20, 2..."


In [ ]:
# 장르 array 만들기
gnr_array = np.zeros((len(train),n_genre))
for i,index in enumerate(train.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(train.loc[index]['plylst_genre_id'])
    for (k,c) in counter.items():
        gnr_array[i][k] = c
gnr_array.shape

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


(115071, 219)

In [ ]:
# 장르 count된 array
gnr_array

array([[ 3.,  3., 10., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 9., 10., 15., ...,  0.,  0.,  0.],
       [22., 27., 86., ...,  0.,  0.,  0.]])

In [ ]:
# 제목 sentencepiece
!pip install sentencepiece

from sentencepiece import SentencePieceProcessor
from sklearn.feature_extraction.text import CountVectorizer

sp = SentencePieceProcessor()
sp.Load("/content/gdrive/My Drive/Colab Notebooks/딥/playlist_title.model")

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000, tokenizer=sp.encode_as_pieces)
# content = train['plylst_title'] + val['plylst_title'] + test['plylst_title'] # train/val/test 추가
content = train['plylst_title']
tdm = cv.fit_transform(content)

title_tdm = tdm.toarray()

     |████████████████████████████████| 1.1MB 2.8MB/s 


In [ ]:
title_tdm # title array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

## Date 정보 가져오기

In [ ]:
# 월 + 일만 가져오기
song['issue_date'] = song['issue_date'].astype('str').map(lambda x:x[:6])

In [ ]:
song.head(1)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],201405,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0


In [ ]:
train.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,num_songs,tags_id,plylst_genre,plylst_genre_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,19,[0],"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...","[0, 1, 2, 3, 4, 5, 6, 4, 7, 2, 3, 4, 7, 2, 3, ..."


In [ ]:
from tqdm import tqdm_notebook
tim_cate = []

for i in tqdm_notebook(range(len(train))):
    tim = [] 
    songs = train.iloc[i,3]
    
    for j in songs:
        tim.append(song.iloc[j,1])
    tim_cate.append(tim)
        
train['plylst_times'] = tim_cate

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
# time id 재부여
plylst_times = train['plylst_times']
times_counter = Counter([tim for times in plylst_times for tim in times])
times_dict = {x: times_counter[x] for x in times_counter}

times_id_tid = dict()
times_tid_id = dict()
for i, t in enumerate(times_dict):
    times_id_tid[t] = i
    times_tid_id[i] = t
n_times = len(times_dict)
train['plylst_times_id'] = train['plylst_times'].map(lambda x: [times_id_tid.get(s) for s in x if times_id_tid.get(s) != None])

In [ ]:
train.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,num_songs,tags_id,plylst_genre,plylst_genre_id,plylst_times,plylst_times_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,19,[0],"[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...","[0, 1, 2, 3, 4, 5, 6, 4, 7, 2, 3, 4, 7, 2, 3, ...","[201305, 201309, 199110, 200009, 200503, 20040...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [ ]:
tim_array = np.zeros((len(train),n_times))
for i,index in enumerate(train.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(train.loc[index]['plylst_times_id'])
    for (k,c) in counter.items():
        tim_array[i][k] = c
tim_array.shape

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


(115071, 844)

In [ ]:
tim_array

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 3., 0., ..., 0., 0., 0.]])

## val, test 전처리
* val

In [ ]:
# 태그 아이디 다시부여
val['tags_id'] = val['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

# len 열 추가
val.loc[:,'num_songs'] = val['songs'].map(len)
val.loc[:,'num_tags'] = val['tags_id'].map(len)

# song_장르 매핑
song_cate = []
for i in range(len(val)):
    gnr = [] 
    songs = val.iloc[i,3]
    for j in songs:
        for k in song.iloc[j,1]:
            gnr.append(k)
    song_cate.append(gnr)
val['plylst_genre'] = song_cate

# 장르 아이디 다시부여
val['plylst_genre_id'] = val['plylst_genre'].map(lambda x: [genre_id_tid.get(s) for s in x if genre_id_tid.get(s) != None])

# 장르 count 행렬 
gnr_val = np.zeros((len(val),n_genre))
for i,index in enumerate(val.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(val.loc[index]['plylst_genre_id'])
    for (k,c) in counter.items():
        gnr_val[i][k] = c
gnr_val.shape

# 타이틀 sentencepiece로 변환
val_title = cv.transform(val['plylst_title']).toarray()


0
10000
20000


In [ ]:
# 날짜 정보 추가 
tim_cate = []

for i in tqdm_notebook(range(len(val))):
    tim = [] 
    songs = val.iloc[i,3]
    
    for j in songs:
        tim.append(song.iloc[j,1])
    tim_cate.append(tim)
    
    
val['plylst_times'] = tim_cate

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
plylst_times = val['plylst_times']
times_counter = Counter([tim for times in plylst_times for tim in times])
times_dict = {x: times_counter[x] for x in times_counter}

times_id_tid = dict()
times_tid_id = dict()
for i, t in enumerate(times_dict):
    times_id_tid[t] = i
    times_tid_id[i] = t
n_timess = len(times_dict)

val['plylst_times_id'] = val['plylst_times'].map(lambda x: [times_id_tid.get(s) for s in x if times_id_tid.get(s) != None])

In [ ]:
val.head(2)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_id,num_songs,num_tags,plylst_genre,plylst_genre_id,plylst_times,plylst_times_id
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,[],27,0,"[2, 0, 1, 5, 0, 1, 2, 0, 1, 5, 0, 3, 2, 0, 1, ...",[],"[201501, 201503, 201508, 201510, 201510, 20160...","[0, 1, 2, 3, 3, 4, 5, 5, 6, 7, 8, 8, 9, 10, 11..."
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[],0,0,[],[],[],[]


In [ ]:
tim_val = np.zeros( (len(val),n_timess) )
for i,index in enumerate(val.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(val.loc[index]['plylst_times_id'])
    for (k,c) in counter.items():
        tim_val[i][k] = c

tim_val.shape

# tim_array = np.zeros((len(train),n_times))
# for i,index in enumerate(train.index):
#     if i%10000 == 0:
#         print(i)
#     counter = Counter(train.loc[index]['plylst_times_id'])
#     for (k,c) in counter.items():
#         tim_array[i][k] = c
# tim_array.shape

0
10000
20000


(23015, 721)

In [ ]:
# val의 user-song matrix 생성 
n_val = len(val)
row = np.repeat(range(n_val), val['num_songs']) # User Index 별 노래 개수만큼 만듦 
col = [song for songs in val['songs'] for song in songs] # Song dic number 추출 
dat = np.repeat(1, val['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
val_user_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_val, n_songs)) # csr_matrix 제작 

row = np.repeat(range(n_val), val['num_tags'])
col = [tag for tags in val['tags_id'] for tag in tags]
dat = np.repeat(1, val['num_tags'].sum())
val_user_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_val, n_tags))

In [ ]:
val_user_songs_A_T = val_user_songs_A.T.tocsr()
val_user_tags_A_T = val_user_tags_A.T.tocsr()

* test

In [ ]:
# 태그 매핑
test['tags_id'] = test['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

test.loc[:,'num_songs'] = test['songs'].map(len)
test.loc[:,'num_tags'] = test['tags_id'].map(len)

# 장르_song 매핑
song_cate = []
for i in range(len(test)):
    gnr = [] 
    songs = test.iloc[i,3]
    
    for j in songs:
        for k in song.iloc[j,1]:
            gnr.append(k)
    song_cate.append(gnr)

test['plylst_genre'] = song_cate
    
# 장르 id 다시매핑
test['plylst_genre_id'] = test['plylst_genre'].map(lambda x: [genre_id_tid.get(s) for s in x if genre_id_tid.get(s) != None])

# 장르 행렬
gnr_test = np.zeros((len(test),n_genre))
for i,index in enumerate(test.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(test.loc[index]['plylst_genre_id'])
    for (k,c) in counter.items():
        gnr_test[i][k] = c
gnr_test.shape

# 제목 변환
test_title = cv.transform(test['plylst_title']).toarray()


0
10000


In [ ]:
# 날짜 가져오기
tim_cate = []

for i in tqdm_notebook(range(len(test))):
    tim = [] 
    songs = test.iloc[i,3]
    
    for j in songs:
        tim.append(song.iloc[j,1])
    tim_cate.append(tim)
    
    
test['plylst_times'] = tim_cate

plylst_times = test['plylst_times']
times_counter = Counter([tim for times in plylst_times for tim in times])
times_dict = {x: times_counter[x] for x in times_counter}

times_id_tid = dict()
times_tid_id = dict()
for i, t in enumerate(times_dict):
    times_id_tid[t] = i
    times_tid_id[i] = t
n_timesss = len(times_dict)
test['plylst_times_id'] = test['plylst_times'].map(lambda x: [times_id_tid.get(s) for s in x if times_id_tid.get(s) != None])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
tim_test = np.zeros((len(test),n_timesss))
for i,index in enumerate(test.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(test.loc[index]['plylst_times_id'])
    for (k,c) in counter.items():
        tim_test[i][k] = c
tim_test.shape

0
10000


(10740, 684)

In [ ]:
# user-song matrix 생성
n_test = len(test)
row = np.repeat(range(n_test), test['num_songs']) # User Index 별 노래 개수만큼 만듦 
col = [song for songs in test['songs'] for song in songs] # Song dic number 추출 
dat = np.repeat(1, test['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
test_user_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_songs)) # csr_matrix 제작 

row = np.repeat(range(n_test), test['num_tags'])
col = [tag for tags in test['tags_id'] for tag in tags]
dat = np.repeat(1, test['num_tags'].sum())
test_user_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_tags))

test_user_songs_A_T = test_user_songs_A.T.tocsr()
test_user_tags_A_T = test_user_tags_A.T.tocsr()

## 모델링에 필요한 행렬 생성 + vstack

In [ ]:
## 1. 장르 + 제목
from scipy import sparse
from scipy.sparse import vstack 

title_gnr = np.concatenate((gnr_array,title_tdm),axis=1)
val_title_gnr = np.concatenate((gnr_val,val_title),axis=1)
test_title_gnr = np.concatenate((gnr_test,test_title),axis=1)

title_gnr = sparse.csr_matrix(title_gnr) # csr matrix로 변환
val_title_gnr = sparse.csr_matrix(val_title_gnr)
test_title_gnr = sparse.csr_matrix(test_title_gnr)

title_gnr = vstack([title_gnr,val_title_gnr,test_title_gnr])

In [ ]:
# 2. song tag train, val, test 합치기(vstack)
from scipy.sparse import vstack 

song_sp = vstack([train_user_songs_A,val_user_songs_A,test_user_songs_A])
tag_sp = vstack([train_user_tags_A,val_user_tags_A,test_user_tags_A])

In [ ]:
song_sp_T = song_sp.T.tocsr()
tag_sp_T = tag_sp.T.tocsr()

In [ ]:
# 3. 날짜 행렬
data_all = pd.concat([train,val,test])
data_all.index = range(len(data_all))

In [ ]:
data_all['num_tags'] = data_all['tags_id'].apply(len)

In [ ]:
# song tim 내용 가져오기. 
tim_cate = []

for i in tqdm_notebook(range(len(data_all))):
    tim = [] 
    songs = data_all.loc[i,'songs']
    for j in songs:
        tim.append(song.iloc[j,1])
    tim_cate.append(tim)
    
    
data_all['plylst_times'] = tim_cate

plylst_times = data_all['plylst_times']
times_counter = Counter([tim for times in plylst_times for tim in times])
times_dict = {x: times_counter[x] for x in times_counter}

times_id_tid = dict()
times_tid_id = dict()
for i, t in enumerate(times_dict):
    times_id_tid[t] = i
    times_tid_id[i] = t
n_times = len(times_dict)
data_all['plylst_times_id'] = data_all['plylst_times'].map(lambda x: [times_id_tid.get(s) for s in x if times_id_tid.get(s) != None])

tim_data_all = np.zeros((len(data_all),n_times))
for i,index in enumerate(data_all.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(data_all.loc[index]['plylst_times_id'])
    for (k,c) in counter.items():
        tim_data_all[i][k] = c
tim_data_all.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000


(148826, 846)

In [ ]:
tim_array = tim_data_all[:len(train)]
tim_val = tim_data_all[len(train):len(train)+len(val)]
tim_test = tim_data_all[len(train)+len(val):len(train)+len(val)+len(test)]

tim_data_all = sparse.csr_matrix(tim_data_all)

tim_array = sparse.csr_matrix(tim_array)
tim_val = sparse.csr_matrix(tim_val)
tim_test = sparse.csr_matrix(tim_test)

times_sp = vstack([tim_array,tim_val,tim_test])

In [ ]:
# 4. 제목
content = data_all['plylst_title']
tdm = cv.fit_transform(content)

title_tdm = tdm.toarray()
title_tr = title_tdm[:len(train)]
title_va = title_tdm[len(train):len(train)+len(val)]
title_ts = title_tdm[len(train)+len(val):len(train)+len(val)+len(test)]

title_sp = sparse.csr_matrix(title_tdm)


## KNN 군집화

In [ ]:
from sklearn.neighbors import NearestNeighbors
aa = 25

# 모델 구축
model_knn_song = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=aa, n_jobs=-1)
model_knn_tag = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=aa, n_jobs=-1)
model_knn_title = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=aa, n_jobs=-1)
model_knn_title_gnr = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=aa, n_jobs=-1)
model_knn_times = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)

# 학습
model_knn_song.fit(song_sp)
model_knn_tag.fit(tag_sp)
model_knn_title.fit(title_sp)
model_knn_title_gnr.fit(title_gnr)
model_knn_times.fit(times_sp)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=25, p=2,
                 radius=1.0)

## CF 모델링
k= 25   
5개 이상 노래만 추천     
인벌스 비사용     

In [ ]:
from tqdm import tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
res = []
for i in tqdm_notebook(range(len(val))):
    data = val.iloc[i]
    pid = i
    aa = 25

    if len(data['songs']) !=0 and len(data['tags_id']) !=0 :
        p = np.zeros((n_songs,1))
        p[data['songs']] = 1

        pp = np.zeros((n_tags,1))
        pp[data['tags_id']] = 1

        tra_song = data_all.iloc[model_knn_song.kneighbors(p.T)[1][0]]   
        row = np.repeat(range(aa), tra_song['num_songs']) # User Index 별 노래 개수만큼 만듦 
        col = [song for songs in tra_song['songs'] for song in songs] # Song dic number 추출 
        dat = np.repeat(1, tra_song['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
        tra_song_sp = spr.csr_matrix((dat, (row, col)), shape=(aa, n_songs)) # csr_matrix 제작 
        tra_song_sp_T = tra_song_sp.T.tocsr()
        
        tra_tag = data_all.iloc[model_knn_tag.kneighbors(pp.T)[1][0]]
        row = np.repeat(range(aa), tra_tag['num_tags'])
        col = [tag for tags in tra_tag['tags_id'] for tag in tags]
        dat = np.repeat(1, tra_tag['num_tags'].sum())
        tra_tag_sp = spr.csr_matrix((dat, (row, col)), shape=(aa, n_tags))
        tra_tag_sp_T = tra_tag_sp.T.tocsr()
        
        tra_tim = times_sp[model_knn_times.kneighbors(tim_val[i:(i+1)])[1][0]]
        tra_title_gnr = title_gnr[model_knn_title_gnr.kneighbors(val_title_gnr[i:(i+1)])[1][0]]
        
        songs_already = data["songs"] 
        tags_already = data["tags_id"]
        
        val_song = cosine_similarity(tra_song_sp,p.T)
        val_tag = cosine_similarity(tra_tag_sp,pp.T)
        val_tim = cosine_similarity(tra_tim,tim_val[i:(i+1)])
        val_title_genre = cosine_similarity(tra_title_gnr,val_title_gnr[i:(i+1)])
        
        vali = val_song * val_tag * val_title_genre * val_tim
        
        cand_song = tra_song_sp_T.dot(vali) # 행에는 노래 열에는 유저 정보 %*% 유사한 유저 -> 유사한 노래에 대하여 높은 값 나옴 
        cand_song_idx = cand_song.reshape(-1).argsort()[-300:][::-1] # 값이 높은 상위 120개 노래 추출

#         cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False]
        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False]  # 중복되는 노래 있는지 확인하고 100개 추출 
        
        ## < inverse coefficient >
        # inv_d = pd.DataFrame(cand_song).loc[cand_song_idx]
        # cand_song_idx = list(pd.DataFrame(inv_d[0] * song_exist.loc[cand_song_idx,'f'])[0].sort_values(ascending=False)[:100].index)
        
        cand_tag = tra_tag_sp_T.dot(vali) # 똑같은 작업 실시 
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-30:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]
        
        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                        })
    
    
    elif len(data['songs']) !=0:
        p = np.zeros((707989,1))
        p[data['songs']] = 1
        
        tra_song = data_all.iloc[model_knn_song.kneighbors(p.T)[1][0]]   
        row = np.repeat(range(aa), tra_song['num_songs']) # User Index 별 노래 개수만큼 만듦 
        col = [song for songs in tra_song['songs'] for song in songs] # Song dic number 추출 
        dat = np.repeat(1, tra_song['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
        tra_song_sp = spr.csr_matrix((dat, (row, col)), shape=(aa, n_songs)) # csr_matrix 제작 
        tra_song_sp_T = tra_song_sp.T.tocsr()
        
        row = np.repeat(range(aa), tra_song['num_tags'])
        col = [tag for tags in tra_song['tags_id'] for tag in tags]
        dat = np.repeat(1, tra_song['num_tags'].sum())
        tra_tag_sp = spr.csr_matrix((dat, (row, col)), shape=(aa, n_tags))
        tra_tag_sp_T = tra_tag_sp.T.tocsr()
        
        songs_already = data["songs"] 
        tags_already = data["tags_id"]

        tra_tim = times_sp[model_knn_times.kneighbors(tim_val[i:(i+1)])[1][0]]        
        tra_title_gnr = title_gnr[model_knn_title_gnr.kneighbors(val_title_gnr[i:(i+1)])[1][0]]
        
        val_song = cosine_similarity(tra_song_sp,p.T)
        val_tim = cosine_similarity(tra_tim,tim_val[i:(i+1)])
        val_title_genre = cosine_similarity(tra_title_gnr,val_title_gnr[i:(i+1)])
        vali = val_song*val_title_genre*val_tim
        
        cand_song = tra_song_sp_T.dot(vali) # 행에는 노래 열에는 유저 정보 %*% 유사한 유저 -> 유사한 노래에 대하여 높은 값 나옴 
        cand_song_idx = cand_song.reshape(-1).argsort()[-300:][::-1] # 값이 높은 상위 120개 노래 추출 

        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False]  # 중복되는 노래 있는지 확인하고 100개 추출 
        
        # inv_d = pd.DataFrame(cand_song).loc[cand_song_idx]
        # cand_song_idx = list(pd.DataFrame(inv_d[0] * song_exist.loc[cand_song_idx,'f'])[0].sort_values(ascending=False)[:100].index)
#         cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]  # 중복되는 노래 있는지 확인하고 100개 추출 

        cand_tag = tra_tag_sp_T.dot(vali) # 똑같은 작업 실시 
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-30:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]
        
        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                    })
        
    elif len(data['tags_id']) !=0:
        p = np.zeros((n_tags,1))
        p[data['tags_id']] = 1
        
        tra_tag = data_all.iloc[model_knn_tag.kneighbors(p.T)[1][0]]   
        row = np.repeat(range(aa), tra_tag['num_songs']) # User Index 별 노래 개수만큼 만듦 
        col = [song for songs in tra_tag['songs'] for song in songs] # Song dic number 추출 
        dat = np.repeat(1, tra_tag['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
        tra_song_sp = spr.csr_matrix((dat, (row, col)), shape=(aa, n_songs)) # csr_matrix 제작 
        tra_song_sp_T = tra_song_sp.T.tocsr()
        
        row = np.repeat(range(aa), tra_tag['num_tags'])
        col = [tag for tags in tra_tag['tags_id'] for tag in tags]
        dat = np.repeat(1, tra_tag['num_tags'].sum())
        tra_tag_sp = spr.csr_matrix((dat, (row, col)), shape=(aa, n_tags))
        tra_tag_sp_T = tra_tag_sp.T.tocsr()
        
        
        songs_already = data["songs"] 
        tags_already = data["tags_id"]
        
        vali = cosine_similarity(tra_tag_sp,pp.T)
        
        if len(data['plylst_title']) != 0 :
            tra_title_gnr = title_tdm[model_knn_title.kneighbors(title_va[i:(i+1)])[1][0]]
            vali_title = cosine_similarity(tra_title_gnr,title_va[i:(i+1)])
            vali = vali * vali_title
                
        cand_song = tra_song_sp_T.dot(vali) # 행에는 노래 열에는 유저 정보 %*% 유사한 유저 -> 유사한 노래에 대하여 높은 값 나옴 
        cand_song_idx = cand_song.reshape(-1).argsort()[-300:][::-1] # 값이 높은 상위 120개 노래 추출 

        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False]  # 중복되는 노래 있는지 확인하고 100개 추출 
        
        # inv_d = pd.DataFrame(cand_song).loc[cand_song_idx]
        # cand_song_idx = list(pd.DataFrame(inv_d[0] * song_exist.loc[cand_song_idx,'f'])[0].sort_values(ascending=False)[:100].index)    
#         cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]  # 중복되는 노래 있는지 확인하고 100개 추출 

        cand_tag = tra_tag_sp_T.dot(vali) # 똑같은 작업 실시 
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-30:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]

        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                    })
        
    else:
        cand_song = []
        for li in data_all.iloc[model_knn_title.kneighbors(title_va[i:(i+1)])[1][0]].songs.to_list():
            for j in li:
                cand_song.append(j)
                
        cand_tag = []
        for li in data_all.iloc[model_knn_title.kneighbors(title_va[i:(i+1)])[1][0]].tags.to_list():
            for j in li:
                cand_tag.append(j)
        
        cand_song_idx = list(pd.DataFrame(cand_song)[0].value_counts()[:100].index)
        rec_tag_idx = list(pd.DataFrame(cand_tag)[0].value_counts()[:10].index)

        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                    })

for i in range(len(res)):
    if len(res[i]['songs']) != 100:
        print('song 에서 {}번째 오류 발생'.format(i))
        
    if len(res[i]['tags']) != 10:
        print('tag 에서 {}번째 오류 발생'.format(i))

import io
import os

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

rec = []
for i in range(len(res)):
    rec.append({
                    "id": res[i]['id'],
                    "songs": list(res[i]['songs']),
                    "tags": res[i]['tags']
                    })
    
write_json(rec, "/content/gdrive/My Drive/Colab Notebooks/머신/82best_date_results.json")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.
